In [1]:
base_dir='dataset/annotations/annotations/tcia-lidc-xml/'

In [2]:
import os

c = 0
# Verify XML file paths
for root_dir, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".xml"):
            c += 1
            # print(os.path.join(root_dir, file))  # Print XML file paths
print(c)


1319


In [4]:
import xml.etree.ElementTree as ET

file_path='dataset/annotations/annotations/tcia-lidc-xml/157/158.xml'

# Parse the XML file
tree = ET.parse(file_path)
root = tree.getroot()

# Print the full XML structure
print(ET.tostring(root, encoding="utf-8").decode("utf-8"))

<ns0:LidcReadMessage xmlns:ns0="http://www.nih.gov" uid="1.3.6.1.4.1.14519.5.2.1.6279.6001.1308168927827.0">
 <ns0:ResponseHeader>
  <ns0:Version>1.7</ns0:Version>
  <ns0:MessageId>1151420</ns0:MessageId>
  <ns0:DateRequest>2006-06-05</ns0:DateRequest>
  <ns0:TimeRequest>17:05:26</ns0:TimeRequest>
  <ns0:RequestingSite>removed</ns0:RequestingSite>
  <ns0:ServicingSite>removed</ns0:ServicingSite>
  <ns0:TaskDescription>Second unblinded read</ns0:TaskDescription>
  <ns0:CtImageFile>removed</ns0:CtImageFile>
  <ns0:SeriesInstanceUid>1.3.6.1.4.1.14519.5.2.1.6279.6001.303494235102183795724852353824</ns0:SeriesInstanceUid>
  <ns0:StudyInstanceUID>1.3.6.1.4.1.14519.5.2.1.6279.6001.339170810277323131167631068432</ns0:StudyInstanceUID>
  <ns0:DateService>2006-06-05</ns0:DateService>
  <ns0:TimeService>17:05:26</ns0:TimeService>
  <ns0:ResponseDescription>1 - Reading complete</ns0:ResponseDescription>
  <ns0:ResponseComments>Merged, reader anonymized, unblinded responses</ns0:ResponseComments>
 

In [15]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Function to parse an XML file
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    nodule_data = []
    
    for reading_session in root.iter():
        if "readingSession" in reading_session.tag.lower():
            for unblinded_nodule in reading_session.iter():
                if "unblindedReadNodule" in unblinded_nodule.tag.lower():
                    
                    # Extract Nodule ID
                    nodule_id = None
                    for child in unblinded_nodule.iter():
                        if "noduleID" in child.tag.lower():
                            nodule_id = child.text.strip() if child.text else "Missing"
                            break
                    nodule_id = nodule_id if nodule_id else "Missing"

                    # Extract Malignancy
                    malignancy = None
                    for child in unblinded_nodule.iter():
                        if "malignancy" in child.tag.lower():
                            malignancy = child.text.strip() if child.text else "Missing"
                            break
                    malignancy = malignancy if malignancy else "Missing"

                    # Extract Bounding Box
                    x_coords = []
                    y_coords = []
                    
                    for edge in unblinded_nodule.iter():
                        if "edgeMap" in edge.tag.lower():
                            x = edge.find("xCoord")
                            y = edge.find("yCoord")
                            if x is not None and y is not None:
                                x_coords.append(int(x.text))
                                y_coords.append(int(y.text))

                    if x_coords and y_coords and len(x_coords) > 1:
                        bbox = [min(x_coords), min(y_coords), max(x_coords), max(y_coords)]
                    else:
                        bbox = "Invalid"
                        print(f"⚠️ EdgeMap missing in file: {file_path}")

                    # Append results
                    nodule_data.append([file_path, nodule_id, malignancy, bbox])

    return nodule_data

# Directory containing XML files
base_dir = "dataset/annotations/annotations/tcia-lidc-xml"

# Collect data from all XML files
all_nodule_data = []
xml_count = 0

for root_dir, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".xml"):
            file_path = os.path.join(root_dir, file)
            xml_count += 1
            print(f"Processing: {file_path}")
            all_nodule_data.extend(parse_xml(file_path))

print(f"Total XML files processed: {xml_count}")

# Convert to DataFrame
df = pd.DataFrame(all_nodule_data, columns=["File", "NoduleID", "Malignancy", "BoundingBox"])
print(df.head())
print(df.shape)


Processing: dataset/annotations/annotations/tcia-lidc-xml\161-resubmitted-correction-3-9-12.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\158.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\159.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\160.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\161.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\162.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\163.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\164.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\165.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\166.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\167.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\157\168.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\185\068.xml
Processing: dataset/annotations/annotations/tcia-lidc-xml\185\06

In [17]:
import os

base_dir = "dataset/annotations/annotations/tcia-lidc-xml"

xml_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(base_dir)
    for file in files
    if file.endswith(".xml")
]

print(f"Total XML files found: {len(xml_files)}")
print("First 5 XML files:", xml_files[:5])

Total XML files found: 1319
First 5 XML files: ['dataset/annotations/annotations/tcia-lidc-xml\\161-resubmitted-correction-3-9-12.xml', 'dataset/annotations/annotations/tcia-lidc-xml\\157\\158.xml', 'dataset/annotations/annotations/tcia-lidc-xml\\157\\159.xml', 'dataset/annotations/annotations/tcia-lidc-xml\\157\\160.xml', 'dataset/annotations/annotations/tcia-lidc-xml\\157\\161.xml']


In [19]:
import xml.etree.ElementTree as ET


def parse_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        print(f"\n✅ Successfully parsed: {file_path}")
        print(
            ET.tostring(root, encoding="utf8").decode("utf8")[:500]
        )  # Print first 500 chars
    except Exception as e:
        print(f"\n❌ Error parsing: {file_path} -> {e}")

In [16]:
df.head(1000)

,File,NoduleID,Malignancy,BoundingBox
